In [1]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.device_count())  # Should return the number of GPUs
print(torch.cuda.get_device_name(0))  # Should show the GPU model

True
1
NVIDIA RTX 5000 Ada Generation


In [2]:
import os

os.environ["HF_HOME"] = "D:/huggingface_cache" 
os.environ["TRANSFORMERS_CACHE"] = "D:/huggingface_cache"
os.environ["HUGGINGFACE_HUB_CACHE"] = "D:/huggingface_cache"


In [3]:
import os

print("HF_HOME:", os.getenv("HF_HOME"))
print("TRANSFORMERS_CACHE:", os.getenv("TRANSFORMERS_CACHE"))
print("HUGGINGFACE_HUB_CACHE:", os.getenv("HUGGINGFACE_HUB_CACHE"))

HF_HOME: D:/huggingface_cache
TRANSFORMERS_CACHE: D:/huggingface_cache
HUGGINGFACE_HUB_CACHE: D:/huggingface_cache


In [ ]:
from huggingface_hub import login
login("#")
# Don't forget to remove the Key when uploading to GitHub

d:\Anaconda\envs\openfl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os


model_name = "meta-llama/Meta-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to("cuda")




d:\Anaconda\envs\openfl\lib\site-packages\transformers\utils\hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [07:11<00:00, 107.78s/it]


In [6]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

prompt = f"""
        Generate {1} question-answer pairs based on the following text segment. 
        Return the result in valid JSON format as a list of objects.

        Text Segment:
        
Wikipedia is a free online encyclopedia that anyone can edit, and millions already have.

Wikipedia's purpose is to benefit readers by presenting information on all branches of knowledge. Hosted by the Wikimedia Foundation, Wikipedia consists of freely editable content, with articles that usually contain numerous links guiding readers to more information.

Written collaboratively by volunteers known as Wikipedians, Wikipedia articles can be edited by anyone with Internet access, except in limited cases in which editing is restricted to prevent disruption or vandalism. Since its creation on January 15, 2001, it has grown into the world's largest reference website, attracting over a billion visitors each month. Wikipedia currently has more than sixty-four million articles in more than 300 languages, including 6,969,912 articles in English, with 125,967 active contributors in the past month.

Wikipedia's fundamental principles are summarized in its five pillars. While the Wikipedia community has developed many policies and guidelines, new editors do not need to be familiar with them before they start contributing.

Anyone can edit Wikipedia's text, data, references, and images. The quality of content is more important than the expertise of who contributes it. Wikipedia's content must conform with its policies, including being verifiable by published reliable sources. Contributions based on personal opinions, beliefs, or personal experiences, unreviewed research, libellous material, and copyright violations are not allowed, and will not remain. Wikipedia's software makes it easy to reverse errors, and experienced editors watch and patrol bad edits.

Wikipedia differs from printed references in important ways. Anyone can instantly improve it, add quality information, remove misinformation, and fix errors and vandalism. Since Wikipedia is continually updated, encyclopedic articles on major news events appear within minutes. 

        Response Format:
        [
            {{"question": "What is ...?", "answer": "The answer is ..."}},
            {{"question": "How does ... work?", "answer": "It works by ..."}}
        ]

        Question answers should be at least 250 words long.

        Do NOT include any explanation or pre-amble before or after the JSON output.
        Return ONLY valid JSON output.  

        Answer:
        """
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    output_tokens = model.generate(**inputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [71]:
generated_tokens = output_tokens[0][len(inputs["input_ids"][0]):]  
generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

In [70]:
generated_text

'I would like the questions to be related to the paragraph, and the answers to be concise and to the point. I would also like the questions to be in the form of a question, and the answers to be in the form of a sentence. I would like the questions and answers to be in English.\n\nI would like the questions and answers to be related to the paragraph, and the answers to be concise and to the point. I would also like the questions to be in the form of a question, and the answers to be in the form of a sentence. I would like the questions and answers to be in English.\n\nI would like the questions and answers to be related to the paragraph, and the answers to be concise and to the point. I would also like the questions to be in the form of a question, and the answers to be in the form of a sentence. I would like the questions and answers to be in English.\n\nI would like the questions and answers to be related to the paragraph, and the answers to be concise and to the point. I would also 

In [68]:
stripped = generated_text.strip()

In [69]:
stripped

'I would like the questions to be related to the paragraph, and the answers to be concise and to the point. I would also like the questions to be in the form of a question, and the answers to be in the form of a sentence. I would like the questions and answers to be in English.\n\nI would like the questions and answers to be related to the paragraph, and the answers to be concise and to the point. I would also like the questions to be in the form of a question, and the answers to be in the form of a sentence. I would like the questions and answers to be in English.\n\nI would like the questions and answers to be related to the paragraph, and the answers to be concise and to the point. I would also like the questions to be in the form of a question, and the answers to be in the form of a sentence. I would like the questions and answers to be in English.\n\nI would like the questions and answers to be related to the paragraph, and the answers to be concise and to the point. I would also 

In [ ]:
# def extract_qa(text, chunk_limit, model="gpt-4", questions_num=5):
#     # Set up data preparation model
#     model_name = "meta-llama/Meta-Llama-3-8B"

#     try:
#         tokenizer = AutoTokenizer.from_pretrained(model_name)
#         model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto") #Use torch.bfloat16 if your GPU supports it.
#     except Exception as e:
#         print(e)
#         return json.dumps({"error": f"The data preparation model was not properly loaded."}, indent=4)

#     text_chunks = split_text(text, max_tokens=4000)  # Adjust chunk size
#     results = []

#     total_chunks = len(text_chunks)

#     for i, chunk in enumerate(text_chunks[:chunk_limit]):
#         print(f'Total {total_chunks}, finished {i + 1}')
